<a href="https://colab.research.google.com/github/AinzOwl/mysticai-colab/blob/main/mystic_ai_sdxl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mystic.Ai SDXL v1 with refiner model deployment script**

Hello, I'm [Ainzoal](https://ainz.sbs) you can find me on mystic.ai discord server, I made this colab after suffering a lot to deploy my first model on v2 to point i quited but now v3 is out and with help of **Paul** i managed to deploy my first model, so i decided to give back and help other people who want to deploy on this super amazing platform.

this colab is really easy, and if you face any errors just ask, anyway i tried to document it and make it as clear as possible so here you go.

For V1.5 [click here](https://github.com/AinzOwl/mysticai-colab/blob/main/mystic_ai_sd1_5.ipynb)

# **Downloading packages**

This was actually the error i faced i had missing packages, but worry not this will fix it just hit the download, also you dont need gpu for this so if it's enabled at top right u can just disable it to save on google resources.

In [ ]:
#@title Download Packages

!pip install pipeline-ai torch transformers diffusers accelerate xformers invisible_watermark safetensors

# **setup api key**

When you run this it will ask you for your api key if you don't know where to find it it's [Here](https://mystic.ai/api-tokens) just type it and hit enter

In [ ]:
#@title Setup Mystic.Ai Api Key

api_key = input('Enter mystic.ai api key: ')
!pipeline cluster login catalyst-api {api_key} -u https://www.mystic.ai -a

# **import packages**
Yeah just run this one and that's it, nothing to change

In [ ]:
#@title Import packages

from pathlib import Path
from typing import List

import torch
from diffusers import DiffusionPipeline

from pipeline import Pipeline, Variable, entity, pipe
from pipeline.cloud import compute_requirements, environments, pipelines
from pipeline.objects import File
from pipeline.objects.graph import InputField, InputSchema


# **inputs (please read)**

this is the configuration for the model inputs, you might want to set up this one, the default values are set a bit high for the model set in this demo, but its easy to change them just search for name of value you want to change and change its default, for example you want to set default number of inference steps to 30 then go to num_inference_steps and change 50 with 30

for negative prompt you could leave it empty by doing "" but as of making this there's a bug that make it required so i suggest typing a generic negative prompt, the one used is one suggested in animagine-xl huggingface.

In [ ]:
#@title Setup inputs

class ModelKwargs(InputSchema):
    num_inference_steps: int | None = InputField(
        title="Number of inference steps",
        default=50,
    )

    guidance_scale: float | None = InputField(
        default=12,
        title="Guidance scale",
        ge=7.5,
        le=12.5,
    )

    denoising_end: float | None = InputField(
        default=0.8,
        title="Denoising end",
        ge=0.0,
        le=1.0,
    )

    negative_prompt: str | None = InputField(
        default='lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry',
        title="Negative Prompt",
    )

    height: int | None = InputField(
        default=1024,
        title="Height",
        ge=64,
        le=1024,
        multiple_of=8,
    )

    width: int | None = InputField(
        default=1024,
        title="Width",
        ge=64,
        le=1024,
        multiple_of=8,
    )



# **Setting up the model**

here you select your model and refiner
just note this uses diffusers so the model must be in huggingface.co.

if it doesn't work then the moddel could be missing something this colab was tried with Linaqruf/animagine-xl and it worked perfectly.

To change the model change ``Linaqruf/animagine-xl`` in the code.

To change the refiner change ``stabilityai/stable-diffusion-xl-refiner-1.0`` in the code


In [ ]:
#@title Setup Model

@entity
class StableDiffusionModel:
    @pipe(on_startup=True, run_once=True)
    def load(self):
        self.base = DiffusionPipeline.from_pretrained(
            "Linaqruf/animagine-xl",
            torch_dtype=torch.float16,
            use_safetensors=True,
        )
        self.base.to("cuda")
        self.refiner = DiffusionPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-refiner-1.0",
            text_encoder_2=self.base.text_encoder_2,
            vae=self.base.vae,
            torch_dtype=torch.float16,
            use_safetensors=True,
            variant="fp16",
        )
        self.refiner.to("cuda")

    @pipe
    def predict(self, prompt: str, kwargs: ModelKwargs) -> List[File]:
        image = self.base(
            prompt=prompt,
            negative_prompt=kwargs.negative_prompt,
            height=kwargs.height,
            width=kwargs.width,
            num_inference_steps=kwargs.num_inference_steps,
            denoising_end=kwargs.denoising_end,
            guidance_scale=kwargs.guidance_scale,
            output_type="latent",
        ).images
        images = self.refiner(
            prompt=prompt,
            negative_prompt=kwargs.negative_prompt,
            num_inference_steps=kwargs.num_inference_steps,
            guidance_scale=kwargs.guidance_scale,
            denoising_start=kwargs.denoising_end,
            image=image,
        ).images

        output_images = []
        for i, image in enumerate(images):
            path = Path(f"/tmp/sd/image-{i}.jpg")
            path.parent.mkdir(parents=True, exist_ok=True)
            image.save(str(path))
            output_images.append(File(path=path, allow_out_of_context_creation=True))

        return output_images

# **Environment**

Env is the one holding the packages that your model uses, the ones selected are the ones needed to get this diffusers model to run and same env can be used for different models so you kinda really just need to run this one ones and you're good

In [ ]:
#@title setup mystic environment

with Pipeline() as builder:
    prompt = Variable(
        str,
        title="Prompt",
    )
    kwargs = Variable(
        ModelKwargs,
        title="Model kwargs",
    )

    model = StableDiffusionModel()

    model.load()

    output = model.predict(prompt, kwargs)

    builder.output(output)

my_pl = builder.get_pipeline()

try:
    environments.create_environment(
        "stable-diffusion-xl-refiner-1.0",
        python_requirements=[
            "torch==2.0.1",
            "transformers==4.30.2",
            "diffusers==0.19.3",
            "accelerate==0.21.0",
            "xformers==0.0.21",
            "invisible_watermark==0.2.0",
            "safetensors==0.3.3",
        ],
    )
except Exception:
    pass

# **Pushing the model**

pushing is easy you just replace ``animagine-xl`` with what you named the model and thats it, if you used a different name for environement you can change it.

For vram it depends on model how much it need but dont overkill it just try to find the balance.

For accelerators check this page [Mystic Ai Docs](https://docs.mystic.ai/docs/gpus-and-accelerators) you'll find the id of each and pick one you want depending on price and vram

In [ ]:
#@title Push the model (open this to change required gpu type)

remote_pipeline = pipelines.upload_pipeline(
    my_pl,
    "animagine-xl:latest",
    environment_id_or_name="stable-diffusion-xl-refiner-1.0",
    required_gpu_vram_mb=20_000,
    accelerators=[
        compute_requirements.Accelerator.nvidia_a100,
    ],
)

print(remote_pipeline.id)